# BladeOne Component Analysis

This notebook analyzes the BladeOne template engine implementation in the WP WooCommerce Printify Sync plugin. It will:
1. Check for missing BladeOne component files
2. Analyze how BladeOne is being used throughout the plugin
3. Generate a report of missing files and usage patterns

## Import Required Libraries

In [ ]:
import os
import glob
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

## Check for Missing BladeOne Component Files

BladeOne is a standalone implementation of Laravel Blade template engine. Let's check for missing core components by defining the expected files and comparing with what's available in our plugin.

In [ ]:
# Define the plugin's root directory
plugin_root = '/home/apolloweb/projects/wp-woocommerce-printify-sync/wp-content/plugins/wp-woocommerce-printify-sync'

# Define expected BladeOne component files
expected_bladeone_files = [
    'BladeOne.php',
    'BladeOneCache.php',
    'BladeOneCustom.php',
    'BladeOneLang.php',
    'BladeOneLogic.php',
    'BladeOneAuth.php',
    'BladeOneHtml.php',
    'BladeOneCsrf.php'
]

# Possible locations where BladeOne could be installed
possible_bladeone_paths = [
    os.path.join(plugin_root, 'vendor/eftec/bladeone'),
    os.path.join(plugin_root, 'includes/lib/bladeone'),
    os.path.join(plugin_root, 'includes/vendor/bladeone'),
    os.path.join(plugin_root, 'lib/bladeone')
]

# Check each possible location
found_bladeone_path = None
for path in possible_bladeone_paths:
    if os.path.exists(path):
        found_bladeone_path = path
        break

if found_bladeone_path:
    print(f"Found BladeOne at: {found_bladeone_path}")
    
    # Get list of actual files
    actual_files = [os.path.basename(f) for f in glob.glob(os.path.join(found_bladeone_path, '*.php'))]
    
    # Find missing files
    missing_files = [f for f in expected_bladeone_files if f not in actual_files]
    
    print("\nMissing BladeOne component files:")
    if missing_files:
        for file in missing_files:
            print(f" - {file}")
    else:
        print("No missing files - All expected BladeOne components are present.")
    
    # Find extra files that might be custom extensions
    extra_files = [f for f in actual_files if f not in expected_bladeone_files]
    
    print("\nExtra BladeOne related files (potential custom extensions):")
    if extra_files:
        for file in extra_files:
            print(f" - {file}")
else:
    print("BladeOne library not found in the expected locations.")

## Analyze BladeOne Library Usage in the Plugin

Next, let's search through the plugin's PHP files to find where and how BladeOne is being used.

In [ ]:
def find_php_files(base_dir):
    """Find all PHP files in the project"""
    return list(Path(base_dir).rglob('*.php'))

def search_for_bladeone_usage(file_path):
    """Search for BladeOne usage in a file"""
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        try:
            content = file.read()
            
            # Look for various BladeOne usage patterns
            blade_instantiation = re.search(r'new\s+(\w*Blade\w*)', content)
            blade_view_calls = re.findall(r'->run\(\s*[\'"]([^\'"]+)[\'"]', content)
            blade_include_statements = re.findall(r'@include\(\s*[\'"]([^\'"]+)[\'"]', content)
            blade_extends_statements = re.findall(r'@extends\(\s*[\'"]([^\'"]+)[\'"]', content)
            blade_imports = re.search(r'use\s+.*Blade\w*', content)
            
            results = {
                'file': str(file_path),
                'instantiation': blade_instantiation.group(1) if blade_instantiation else None,
                'view_calls': blade_view_calls,
                'includes': blade_include_statements,
                'extends': blade_extends_statements,
                'imports': blade_imports.group(0) if blade_imports else None,
                'has_usage': bool(blade_instantiation or blade_view_calls or 
                                  blade_include_statements or blade_extends_statements or 
                                  blade_imports)
            }
            return results
        except Exception as e:
            return {
                'file': str(file_path),
                'error': str(e),
                'has_usage': False
            }

# Find all PHP files
php_files = find_php_files(plugin_root)
print(f"Found {len(php_files)} PHP files to analyze")

# Analyze each file for BladeOne usage
results = []
for file in php_files:
    result = search_for_bladeone_usage(file)
    if result.get('has_usage'):
        results.append(result)
        
print(f"Found {len(results)} files with BladeOne usage")

# Convert to DataFrame for easier analysis
usage_df = pd.DataFrame(results)
if not usage_df.empty:
    display(usage_df[['file', 'instantiation', 'imports']].head())

In [ ]:
# Find template files
def find_blade_templates(base_dir):
    """Find all blade template files"""
    return list(Path(base_dir).rglob('*.blade.php'))

blade_templates = find_blade_templates(plugin_root)
print(f"Found {len(blade_templates)} Blade template files")

# Check common template directories
template_dirs = [
    os.path.join(plugin_root, 'templates'),
    os.path.join(plugin_root, 'views'),
    os.path.join(plugin_root, 'resources/views')
]

for template_dir in template_dirs:
    if os.path.exists(template_dir):
        print(f"Template directory found: {template_dir}")
        template_files = list(Path(template_dir).rglob('*.php'))
        print(f" - Contains {len(template_files)} PHP files")
        
        # Analyze some templates for blade directives
        if template_files:
            blade_directives_count = {}
            for template in template_files[:10]:  # Look at first 10 files
                try:
                    with open(template, 'r', encoding='utf-8', errors='ignore') as file:
                        content = file.read()
                        directives = re.findall(r'@(\w+)', content)
                        for directive in directives:
                            if directive in blade_directives_count:
                                blade_directives_count[directive] += 1
                            else:
                                blade_directives_count[directive] = 1
                except Exception:
                    pass
            
            print(" - Blade directives used:")
            for directive, count in sorted(blade_directives_count.items(), key=lambda x: x[1], reverse=True):
                print(f"   * @{directive}: {count} uses")

## Generate a Report of Missing Files and Usage

Let's create a comprehensive report of our findings.

In [ ]:
# Create a report dataframe
report_data = {
    'category': [],
    'item': [],
    'status': [],
    'details': []
}

# Add BladeOne library status
if found_bladeone_path:
    report_data['category'].append('Library')
    report_data['item'].append('BladeOne')
    report_data['status'].append('Found')
    report_data['details'].append(f'Location: {found_bladeone_path}')
    
    # Add missing files
    for file in missing_files:
        report_data['category'].append('Required File')
        report_data['item'].append(file)
        report_data['status'].append('Missing')
        report_data['details'].append('Required BladeOne component file not found')
else:
    report_data['category'].append('Library')
    report_data['item'].append('BladeOne')
    report_data['status'].append('Missing')
    report_data['details'].append('BladeOne library not found in expected locations')

# Add usage stats
if not usage_df.empty:
    total_usage = len(usage_df)
    report_data['category'].append('Usage')
    report_data['item'].append('Files Using BladeOne')
    report_data['status'].append('Found')
    report_data['details'].append(f'{total_usage} files using BladeOne components')

# Add template stats
report_data['category'].append('Templates')
report_data['item'].append('Blade Templates')
report_data['status'].append('Found' if blade_templates else 'Not Found')
report_data['details'].append(f'{len(blade_templates)} blade template files found')

# Create the report DataFrame
report_df = pd.DataFrame(report_data)
display(report_df)

# Save the report to a CSV file
report_path = os.path.join(plugin_root, 'bladeone_analysis_report.csv')
report_df.to_csv(report_path, index=False)
print(f"Report saved to {report_path}")

## Visualize the Findings

In [ ]:
# Set the style
plt.style.use('ggplot')

# Create a figure with subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Status of BladeOne components (if found_bladeone_path exists)
if found_bladeone_path:
    component_status = {
        'Present': len(expected_bladeone_files) - len(missing_files),
        'Missing': len(missing_files)
    }
    
    ax1.bar(component_status.keys(), component_status.values(), color=['green', 'red'])
    ax1.set_title('BladeOne Component Status')
    ax1.set_ylabel('Number of Components')
    
    for i, v in enumerate(component_status.values()):
        ax1.text(i, v + 0.1, str(v), ha='center')
else:
    ax1.text(0.5, 0.5, 'BladeOne Library Not Found', 
             ha='center', va='center', fontsize=14, color='red')
    ax1.axis('off')

# Plot 2: Files using BladeOne
if not usage_df.empty:
    # Count files by type of usage
    usage_counts = {
        'Instantiations': usage_df['instantiation'].notna().sum(),
        'View Calls': sum(len(calls) > 0 for calls in usage_df['view_calls']),
        'Includes': sum(len(incs) > 0 for incs in usage_df['includes']),
        'Extends': sum(len(exts) > 0 for exts in usage_df['extends'])
    }
    
    ax2.bar(usage_counts.keys(), usage_counts.values(), color='blue')
    ax2.set_title('BladeOne Usage in Files')
    ax2.set_ylabel('Number of Files')
    
    for i, v in enumerate(usage_counts.values()):
        ax2.text(i, v + 0.1, str(v), ha='center')
else:
    ax2.text(0.5, 0.5, 'No BladeOne Usage Found', 
             ha='center', va='center', fontsize=14, color='red')
    ax2.axis('off')

plt.tight_layout()
plt.show()

# Save the visualization
visualization_path = os.path.join(plugin_root, 'bladeone_analysis_visualization.png')
fig.savefig(visualization_path)
print(f"Visualization saved to {visualization_path}")

## Recommendations for Missing BladeOne Components

Based on the analysis, here are recommendations for handling any missing BladeOne components:

In [ ]:
def generate_recommendations():
    """Generate recommendations based on analysis"""
    recommendations = []
    
    if not found_bladeone_path:
        recommendations.append({
            'issue': 'Missing BladeOne Library',
            'recommendation': 'Install BladeOne via Composer: composer require eftec/bladeone',
            'priority': 'High'
        })
    else:
        if missing_files:
            for file in missing_files:
                recommendations.append({
                    'issue': f'Missing Component: {file}',
                    'recommendation': 'Download the missing file from the official BladeOne repository or reinstall the library',
                    'priority': 'Medium'
                })
    
    # Check template directories
    template_dir_exists = any(os.path.exists(dir) for dir in template_dirs)
    if not template_dir_exists:
        recommendations.append({
            'issue': 'No Standard Template Directory',
            'recommendation': 'Create a "views" or "templates" directory for storing blade templates',
            'priority': 'Low'
        })
    
    # Check for compiled template directory
    compiled_templates_dir = os.path.join(plugin_root, 'cache/views')
    if not os.path.exists(compiled_templates_dir):
        recommendations.append({
            'issue': 'Missing Compiled Templates Directory',
            'recommendation': 'Create a writable "cache/views" directory for storing compiled templates',
            'priority': 'Medium'
        })
        
    return pd.DataFrame(recommendations)

recommendations_df = generate_recommendations()
if not recommendations_df.empty:
    display(recommendations_df)
else:
    print("No recommendations needed - BladeOne appears to be properly configured.")

# Save recommendations to a CSV
if not recommendations_df.empty:
    recommendations_path = os.path.join(plugin_root, 'bladeone_recommendations.csv')
    recommendations_df.to_csv(recommendations_path, index=False)
    print(f"Recommendations saved to {recommendations_path}")

## Conclusion

This analysis has:
1. Checked for the presence of BladeOne library files in the plugin
2. Identified any missing core BladeOne components
3. Analyzed how BladeOne is being used throughout the plugin code
4. Identified all Blade template files and their usage patterns
5. Generated recommendations for fixing any issues found

The reports generated by this notebook can be used to ensure the plugin's template system is working correctly and help identify any missing components that need to be installed.